In [9]:
# --- Core imports ---
import os
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path
from gfpgan import GFPGANer

# --- Display info ---
print("✅ Environment ready")


✅ Environment ready


In [10]:
# === PATH CONFIGURATION ===
SRC_DIR  = Path(r"../data/crops_face/20251107/1_cafe_pos_person_faces_yunet")
OUT_DIR  = Path(r"../data/crops_face/20251107/2_cafe_pos_faces_realistic")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# === MODEL FILES ===
FSRCNN_MODEL = r"../models/FSRCNN_x4.pb"
GFPGAN_MODEL = r"../models/GFPGANv1.4.pth"

# === UPSCALE SETTINGS ===
UPSCALE_FACTOR = 4  # FSRCNN default x4
FACE_UPSCALE = 2    # GFPGAN internal upscale

print(f"📂 Input folder: {SRC_DIR}")
print(f"📦 Output folder: {OUT_DIR}")
print(f"🔧 Using FSRCNN: {FSRCNN_MODEL}")
print(f"🤖 Using GFPGAN: {GFPGAN_MODEL}")


📂 Input folder: ..\data\crops_face\20251107\1_cafe_pos_person_faces_yunet
📦 Output folder: ..\data\crops_face\20251107\2_cafe_pos_faces_realistic
🔧 Using FSRCNN: ../models/FSRCNN_x4.pb
🤖 Using GFPGAN: ../models/GFPGANv1.4.pth


In [11]:
print("⏳ Loading FSRCNN super-resolution model...")
sr = cv2.dnn_superres.DnnSuperResImpl_create()
sr.readModel(FSRCNN_MODEL)
sr.setModel("fsrcnn", UPSCALE_FACTOR)
print("✅ FSRCNN loaded successfully!")

⏳ Loading FSRCNN super-resolution model...
✅ FSRCNN loaded successfully!


In [12]:
# --- Load GFPGAN model ---
print("⏳ Loading GFPGAN face restoration model...")
restorer = GFPGANer(
    model_path=GFPGAN_MODEL,
    upscale=FACE_UPSCALE,
    arch="clean",
    channel_multiplier=2,
    bg_upsampler=None
)
print("✅ GFPGAN loaded successfully!")

⏳ Loading GFPGAN face restoration model...


✅ GFPGAN loaded successfully!


In [13]:
import os
import cv2
import numpy as np
from pathlib import Path

def enhance_image_humanlike(
    img_path,
    out_dir,
    sr=None,                # FSRCNN super-res model (optional)
    restorer=None,          # GFPGAN restorer (optional)
    use_fsrcnn=True,
    use_gfpgan=False,
    blend_alpha=0.45,
    sharpen_strength=0.08,
    subtle_mode=False
):
    """
    🎯 Flexible CCTV restoration pipeline — realistic, natural results.
    Automatically handles FSRCNN + GFPGAN (either or both).
    """

    try:
        img = cv2.imread(str(img_path))
        if img is None or img.size == 0:
            print(f"⚠️ Failed to read {img_path}")
            return None

        h, w = img.shape[:2]
        small_face_mode = (h < 600 or w < 400)
        base = img.copy()
        restored = base.copy()  # ✅ Ensure variable is always defined

        # --- Step 1️⃣: FSRCNN Super-Resolution (if enabled) ---
        if use_fsrcnn and sr is not None:
            try:
                upscaled = sr.upsample(img)
                upscaled = cv2.GaussianBlur(upscaled, (3, 3), sigmaX=0.4)
                base = upscaled
            except Exception as e:
                print(f"⚠️ FSRCNN failed for {img_path}: {e}")
                base = img.copy()

        # --- Step 2️⃣: GFPGAN Restoration (if enabled) ---
        if use_gfpgan and restorer is not None:
            try:
                _, _, restored = restorer.enhance(
                    base,
                    has_aligned=False,
                    only_center_face=False,
                    paste_back=True
                )
                if restored is None or not isinstance(restored, np.ndarray):
                    print(f"⚠️ GFPGAN returned invalid output for {img_path}")
                    restored = base.copy()
            except Exception as e:
                print(f"⚠️ GFPGAN error for {img_path}: {e}")
                restored = base.copy()
        else:
            restored = base.copy()

        # --- Step 3️⃣: Ensure dimensions and channels match ---
        if restored.shape[:2] != base.shape[:2]:
            #print(f"⚠️ Shape mismatch for {Path(img_path).name}: base={base.shape}, restored={restored.shape}")
            restored = cv2.resize(restored, (base.shape[1], base.shape[0]), interpolation=cv2.INTER_CUBIC)
        if len(restored.shape) == 2:  # grayscale fallback
            restored = cv2.cvtColor(restored, cv2.COLOR_GRAY2BGR)

        # --- Step 4️⃣: Blend Results ---
        alpha = blend_alpha if not small_face_mode else min(blend_alpha + 0.1, 0.6)
        try:
            blended = cv2.addWeighted(base, 1 - alpha, restored, alpha, 0)
        except Exception as e:
            print(f"⚠️ Blending failed for {img_path}: {e}")
            blended = restored.copy()

        # --- Step 5️⃣: Gentle Sharpening (optional) ---
        if subtle_mode and sharpen_strength > 0:
            blurred = cv2.GaussianBlur(blended, (0, 0), 2)
            final = cv2.addWeighted(blended, 1 + sharpen_strength, blurred, -sharpen_strength, 0)
        else:
            final = blended

        # --- Step 6️⃣: Bilateral Filter (natural smoothing) ---
        final = cv2.bilateralFilter(final, d=7, sigmaColor=25, sigmaSpace=25)
        final = np.clip(final, 0, 255).astype(np.uint8)

        # --- Step 7️⃣: Save output ---
        os.makedirs(out_dir, exist_ok=True)
        out_path = os.path.join(out_dir, f"{Path(img_path).stem}.png")
        cv2.imwrite(out_path, final, [int(cv2.IMWRITE_PNG_COMPRESSION), 3])

        return out_path

    except Exception as e:
        print(f"❌ Error processing {img_path}: {e}")
        return None


In [14]:
from tqdm import tqdm
from pathlib import Path

images = sorted(SRC_DIR.glob("*.png")) + sorted(SRC_DIR.glob("*.jpg")) + sorted(SRC_DIR.glob("*.jpeg"))
print(f"📂 Found {len(images)} images to enhance.\n")

for img_path in tqdm(images, desc="Enhancing (Realistic Human-like)"):
    enhance_image_humanlike(
        img_path=img_path,
        out_dir=OUT_DIR,
        sr=sr,
        restorer=restorer,
        blend_alpha=0.5,       # lower = more natural (less GFPGAN)
        sharpen_strength=0.08, # subtle, realistic detail
        subtle_mode=False
    )

print(f"\n✅ All enhanced images saved to: {OUT_DIR}")


📂 Found 7500 images to enhance.



Enhancing (Realistic Human-like): 100%|██████████| 7500/7500 [43:16<00:00,  2.89it/s]


✅ All enhanced images saved to: ..\data\crops_face\20251107\2_cafe_pos_faces_realistic


In [15]:
import os
import cv2
from tqdm import tqdm

def detect_frontal_faces(
    input_dir="../data/crops_face/20251107/2_cafe_pos_faces_realistic",
    output_dir="../data/crops_face/20251107/3_cafe_pos_frontal_faces",
    cascade_path="../models/haarcascade_frontalface_alt2.xml"
):
    """
    Detects and saves only frontal faces using Haar Cascade.
    Keeps images where a frontal face is detected, skips the rest.
    """

    os.makedirs(output_dir, exist_ok=True)

    # Load Haar Cascade
    face_cascade = cv2.CascadeClassifier(cascade_path)
    if face_cascade.empty():
        raise IOError(f"❌ Could not load Haar Cascade: {cascade_path}")

    # Get all image files
    images = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    print(f"📁 Found {len(images)} images to process.")

    kept, skipped = 0, 0

    for img_name in tqdm(images, desc="Detecting frontal faces"):
        img_path = os.path.join(input_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            skipped += 1
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Haar detection (default, sensitive frontal-face detector)
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=2,
            minSize=(20, 20)
        )

        # If any frontal face detected → keep
        if len(faces) > 0:
            out_path = os.path.join(output_dir, img_name)
            cv2.imwrite(out_path, img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
            kept += 1
        else:
            skipped += 1

    print(f"\n✅ Done! {kept} frontal faces saved to: {output_dir}")
    print(f"🚫 Skipped {skipped} images without frontal detections.")


In [16]:
detect_frontal_faces(
    input_dir="../data/crops_face/20251107/2_cafe_pos_faces_realistic",
    output_dir="../data/crops_face/20251107/3_cafe_pos_frontal_faces"
)


📁 Found 7500 images to process.


Detecting frontal faces: 100%|██████████| 7500/7500 [34:36<00:00,  3.61it/s]


✅ Done! 5732 frontal faces saved to: ../data/crops_face/20251107/3_cafe_pos_frontal_faces
🚫 Skipped 1768 images without frontal detections.
